In [25]:
%matplotlib inline
from pyspark.mllib.tree import RandomForest
import xgboost as xgb
from pyspark.mllib.util import MLUtils
from pyspark.mllib.tree import GradientBoostedTrees
import matplotlib
import numpy as np
from operator import add
import matplotlib.pyplot as plt
from pyspark.mllib.regression import LabeledPoint
f=sc.textFile("train.csv")
trainRecord=f.filter(lambda x:"." in x).map(lambda x:x.split(",")).map(lambda x:[x[a] for a in [0,1,2,5,6,7,8,10,12]])
testRecord=sc.textFile("test.csv").filter(lambda x:"." in x).map(lambda x:x.split(",")).map(lambda x:[x[a] for a in [0,1,4,5,6,7,9,11]])
def trainStand(l):
#     gender
    l[3]=[0,1] if l[3]=="male" else [1,0]
#     class
    if l[-1]!="":
        l[-1]=l[-1].replace('S',"0").replace("C","1").replace("Q","2")
    else:
        l[-1]="0"
    vec=np.zeros(3)
    vec[int(l[-1])]=1
    l=l[:3]+l[3]+l[4:-1]+(vec.tolist())
    l[5]="30" if l[5]=="" else l[5]
    return l
def testStand(l):
#     gender
    l[2]=[0,1] if l[2]=="male" else [1,0]
#     class
    if l[-1]!="":
        l[-1]=l[-1].replace('S',"0").replace("C","1").replace("Q","2")
    else:
        l[-1]="0"
    vec=np.zeros(3)
    vec[int(l[-1])]=1
    l=l[:2]+l[2]+l[3:-1]+(vec.tolist())
    l[7]=7.5 if l[7]=="" else l[7]
    l[4]="30" if l[4]=="" else l[4]
    return l
train=trainRecord.map(trainStand).map(lambda x:[float(a) for a in x]).map(lambda x:LabeledPoint(x[1],x[2:]))
test=testRecord.map(testStand).map(lambda x:[float(a) for a in x]).map(lambda x:LabeledPoint(x[0],x[1:]))
# ageTrain=oktrain.filter(lambda x:x[5]!="").map(lambda x:[float(a) for a in x]).map(lambda x:LabeledPoint(x[5],x[2:5]+x[6:]))
# ageTest=oktrain.filter(lambda x:x[5]=="").map(lambda x:[float(a) for a in x if a!=""]).map(lambda x:LabeledPoint(x[0],x[2:]))
# tageTrain=oktest.filter(lambda x:x[4]!="").map(lambda x:[float(a) for a in x]).map(lambda x:LabeledPoint(x[4],x[1:4]+x[5:]))
# tageTest=oktest.filter(lambda x:x[4]=="").map(lambda x:[float(a) for a in x if a!=""]).map(lambda x:LabeledPoint(x[0],x[1:]))

In [33]:
t=train
data=np.array(t.map(lambda x:x.features).collect())
label=np.array(t.map(lambda x:x.label).collect())
xgtrain = xgb.DMatrix( data, label=label)
t=test
data=np.array(t.map(lambda x:x.features).collect())
label=np.array(t.map(lambda x:x.label).collect())
xgtest = xgb.DMatrix( data, label=label)

In [38]:
import xgboost as xgb
from pyspark.mllib.util import MLUtils
t,tt=train.randomSplit([0.7,0.3])
data=np.array(t.map(lambda x:x.features).collect())
label=np.array(t.map(lambda x:x.label).collect())
xgtrain = xgb.DMatrix( data, label=label)
t=tt
data=np.array(t.map(lambda x:x.features).collect())
label=np.array(t.map(lambda x:x.label).collect())
xgtest = xgb.DMatrix( data, label=label)
params={
'booster':'gbtree',
# 这里手写数字是0-9，是一个多类的问题，因此采用了multisoft多分类器，
'objective': 'binary:logistic', 
'subsample':1,
'gamma':0.5,  # 在树的叶子节点下一个分区的最小损失，越大算法模型越保守 。[0:]
'max_depth':3, # 构建树的深度 [1:]
#'lambda':450,  # L2 正则项权重
'colsample_bytree':0.8, # 构建树树时的采样比率 (0:1]
'min_child_weight':9,
'eta': 0.1, # 如同学习率
'seed':710,
}
plst = list(params.items())

# return 训练和验证的错误率
# watchlist = [(xgtrain, 'train')]
watchlist = [(xgtrain, 'train'),(xgtest, 'val')]
num_rounds=3000
# xgb.cv
# training model 
# early_stopping_rounds 当设置的迭代次数较大时，early_stopping_rounds 可在一定的迭代次数内准确率没有提升就停止训练
model = xgb.cv(plst, xgtrain, num_rounds, watchlist,early_stopping_rounds=100)
# cvresult=xgboost.cv(params, xgtrain, num_boost_round=100, nfold=5,
#                 metrics={'error'}, seed=0)
#model.save_model('./model/xgb.model') # 用于存储训练出的模型
preds = model.predict(xgtest,ntree_limit=model.best_iteration)
# def prob2bool(l):
#     if l>0.5:
#         return 1
#     return 0
# result=sc.parallelize(preds).map(prob2bool).zipWithIndex().map(lambda x: (str(x[1]+892)+","+str(x[0])+"\n"))
# a=result.collect()
# file_object = open('thefile4.csv', 'w')
# file_object.write("PassengerId,Survived\n")
# file_object.writelines(a)
# file_object.close( )
# result.count()

TypeError: unsupported operand type(s) for /: 'int' and 'list'

In [37]:
cvresult

{'test-error-mean': [0.20224700000000001,
  0.20224699999999998,
  0.1988762,
  0.20337060000000001,
  0.1988762,
  0.1988762,
  0.1977526,
  0.20112340000000001,
  0.1988762,
  0.19438179999999999,
  0.1988762,
  0.1955054,
  0.1955054,
  0.1955054,
  0.1955054,
  0.1955054,
  0.196629,
  0.1977526,
  0.1988762,
  0.19999980000000001,
  0.1977526,
  0.196629,
  0.1977526,
  0.196629,
  0.19438179999999999,
  0.19213459999999999,
  0.19213459999999999,
  0.19101099999999999,
  0.19101100000000001,
  0.1831458,
  0.18089859999999999,
  0.17752780000000001,
  0.17528060000000001,
  0.17865139999999999,
  0.17977499999999996,
  0.17977499999999996,
  0.18089859999999999,
  0.18089859999999999,
  0.18089859999999999,
  0.18089859999999999,
  0.1831458,
  0.18426939999999997,
  0.18202220000000002,
  0.17977500000000002,
  0.17977500000000002,
  0.17977500000000002,
  0.17977500000000002,
  0.18089859999999999,
  0.18089859999999999,
  0.1831458,
  0.1831458,
  0.1831458,
  0.18426940000000

In [15]:
def prob2bool(l):
    if l>0.5:
        return 1
    return 0
result=sc.parallelize(preds).map(prob2bool).zipWithIndex().map(lambda x: (str(x[1]+892)+","+str(x[0])+"\n"))
a=result.collect()
file_object = open('thefile4.csv', 'w')
file_object.write("PassengerId,Survived\n")
file_object.writelines(a)
file_object.close( )
result.count()

418

In [9]:
sc.parallelize(preds).map(prob2bool).take(15)

[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1]

In [10]:
sc.parallelize(preds).map(prob2bool).take(15)

[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1]

In [11]:
sc.parallelize(preds).map(prob2bool).take(15)

[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1]